In [41]:
import fiona
import math
import json
import uuid
from fiona.crs import from_epsg
from shapely.geometry import shape, MultiPolygon
from shapely.ops import transform, unary_union
import pyproj

## Private ownership

Merge features that share the same owner

In [ ]:
with fiona.open('sources/private-industrial-ownership-wgs84.gpkg', 'r') as src:
    private_industrial = [feature for feature in src]
    
    for feature in private_industrial: 
        if feature['properties']['LandManager'] == 'PVI':
            feature['properties'] = {'owner': feature['properties']['Owner'], 'owner_class': 'private_industrial'}
        else: 
            feature['properties'] = {'owner': 'Private Non-Industrial', 'owner_class': 'private_non_industrial'}
            
    grouped_features = {}
    # combine the features that have the same owner into a single multipolygon
    for feature in private_industrial:
        if feature['properties']['owner'] not in grouped_features:
            grouped_features[feature['properties']['owner']] = feature
        else:
            geometry = shape(feature['geometry'])
            geometry2 = shape(grouped_features[feature['properties']['owner']]['geometry'])
            combined_geometry = unary_union([geometry, geometry2])
            assert type(combined_geometry) == MultiPolygon
            grouped_features[feature['properties']['owner']] = {'geometry': combined_geometry, 'properties': feature['properties']}

Now we add an ID, calculate the area in acres by converting to Oregon Lambert, and calclate the percent of the class each feature represents.

In [ ]:
total_industrial_area = 0
total_non_industrial_area = 0    

for feature in grouped_features.values():
    
    # add a unique id to each feature
    feature['id'] = uuid.uuid4().int % (10**10)
    
    geometry = shape(feature['geometry'])
    # Project from WGS84 (EPSG:4326) to Oregon Lambert (EPSG:2992)
    transformer = pyproj.Transformer.from_crs('EPSG:4326', 'EPSG:2992', always_xy=True)
    projected = transform(transformer.transform, geometry)
    # oregon lambert is in square feet, convert to acres (1 acre = 43,560 square feet)
    feature['properties']['area'] = round(shapely.area(projected) / 43560)
    
    # calculate the total area of each class
    if feature['properties']['owner_class'] == 'private_industrial':
        total_industrial_area += feature['properties']['area']
    else:
        total_non_industrial_area += feature['properties']['area']
        
# add the percent of each class each feature represents
for feature in grouped_features.values():
    if feature['properties']['owner_class'] == 'private_industrial':
        feature['properties']['percent_of_class'] = round(feature['properties']['area'] / total_industrial_area * 100, 2)
    else:
        feature['properties']['percent_of_class'] = round(feature['properties']['area'] / total_non_industrial_area * 100, 2)
        
    feature['properties']['name'] = feature['properties']['owner']
        
        
# print total acres and percent for each feature
for feature in grouped_features.values():
    print(feature['properties']['owner'], feature['properties']['area'], feature['properties']['percent_of_class'])

print(total_industrial_area)
print(total_non_industrial_area)


In [ ]:
schema = {
    'geometry': 'MultiPolygon',
    'id': 'int',
    'properties': {
        'owner': 'str',
        'owner_class': 'str',
        'name': 'str',
        'area': 'int',
        'percent_of_class': 'float'
    }
}

with fiona.open('sources/clean-private-lands.gpkg', 'w', driver='GPKG', schema=schema, crs=from_epsg(4326)) as dst:
    dst.writerecords(grouped_features.values())

Create a summary statistics file

In [ ]:
out_json = {
    'industrial_owners': [],
    'non_industrial_owners': [],
    'total_industrial_acres': total_industrial_area,
    'total_non_industrial_acres': total_non_industrial_area
}

# sort the features by area in descending order
grouped_features_sorted = sorted(grouped_features.values(), key=lambda x: x['properties']['area'], reverse=True)

# add the features to the out_json
for feature in grouped_features_sorted:
    if feature['properties']['owner_class'] == 'private_industrial':
        out_json['industrial_owners'].append({'owner': feature['properties']['owner'], 'acres': feature['properties']['area'], 'percent_of_class': feature['properties']['percent_of_class'], 'id': feature['id']})
    else:
        out_json['non_industrial_owners'].append({'owner': feature['properties']['owner'], 'acres': feature['properties']['area'], 'percent_of_class': feature['properties']['percent_of_class'], 'id': feature['id']})

# save the out_json to a json file
with open('summary/private-ownership.json', 'w') as f:
    json.dump(out_json, f)


# State lands

Merge ODF/state land board owned lands 

In [ ]:
def get_area_acres(geometry):
    transformer = pyproj.Transformer.from_crs('EPSG:4326', 'EPSG:2992', always_xy=True)
    projected = transform(transformer.transform, geometry)
    return round(shapely.area(projected) / 43560)

with fiona.open('sources/ODF-lands.gpkg', 'r', layer='bounds') as src:
    odf_lands = [feature for feature in src]
    geometries = []
    for feature in odf_lands:
        geometry = shape(feature['geometry'])
        geometries.append(geometry)
    odf_lands_feature = {'geometry': unary_union(geometries)}
    odf_lands_feature['properties'] = {'owner': 'Oregon Department of Forestry', 'owner_class': 'state', 'area': get_area_acres(odf_lands_feature['geometry']), 'percent_of_class': 100, 'name': 'Oregon Department of Forestry'}
    odf_lands_feature['id'] = uuid.uuid4().int % (10**10)
    
with fiona.open('sources/oregon-state-land-board.gpkg', 'r', layer='bounds') as src:
    oslb_lands = [feature for feature in src]
    geometries = []
    for feature in oslb_lands:
        geometry = shape(feature['geometry'])
        geometries.append(geometry)
    oslb_lands_feature = {'geometry': unary_union(geometries)}
    oslb_lands_feature['properties'] = {'owner': 'Oregon State Land Board', 'owner_class': 'state', 'area': get_area_acres(oslb_lands_feature['geometry']), 'percent_of_class': 100, 'name': 'Oregon State Land Board'}
    oslb_lands_feature['id'] = uuid.uuid4().int % (10**10)

schema = {
    'geometry': 'MultiPolygon',
    'id': 'int',
    'properties': {
        'owner': 'str',
        'owner_class': 'str',
        'area': 'int',
        'percent_of_class': 'float',
        'name': 'str'
    }
}   

with fiona.open('sources/clean-state-lands.gpkg', 'w', driver='GPKG', schema=schema, crs=from_epsg(4326)) as dst:
    # add the odf_lands_feature and oslb_lands_feature to the dst
    dst.write(odf_lands_feature)
    dst.write(oslb_lands_feature)




## National forests 

Clean up the national forest features

In [ ]:
with fiona.open('sources/usfs-oregon.gpkg', 'r', layer='bounds') as src:
    forest_lands = [feature for feature in src]

    total_forest_area = 0
    for feature in forest_lands:
        area = get_area_acres(shape(feature['geometry']))  
        total_forest_area += area
        feature['properties'] = {'owner': 'United States Forest Service', 'owner_class': 'federal', 'area': area, 'percent_of_class': 0, 'name': feature['properties']['FORESTNAME']}
        feature['id'] = uuid.uuid4().int % (10**10)
        
    for feature in forest_lands:
        feature['properties']['percent_of_class'] = round(feature['properties']['area'] / total_forest_area * 100, 2)
        
        
    schema = {
        'geometry': 'MultiPolygon',
        'id': 'int',
        'properties': {
            'owner': 'str',
            'owner_class': 'str',
            'area': 'int',          
            'percent_of_class': 'float',
            'name': 'str'
        }
    }
    
    with fiona.open('sources/clean-usfs-lands.gpkg', 'w', driver='GPKG', schema=schema, crs=from_epsg(4326)) as dst:
        dst.writerecords(forest_lands)

/var/folders/8h/kk4k5t952ydbc8hfg6nct2940000gn/T/ipykernel_38736/4215322853.py:8: FionaDeprecationWarning: instances of this class -- CRS, geometry, and feature objects -- will become immutable in fiona version 2.0
  feature['properties'] = {'owner': 'United States Forest Service', 'owner_class': 'federal', 'area': area, 'percent_of_class': 0, 'name': feature['properties']['FORESTNAME']}
/var/folders/8h/kk4k5t952ydbc8hfg6nct2940000gn/T/ipykernel_38736/4215322853.py:9: FionaDeprecationWarning: instances of this class -- CRS, geometry, and feature objects -- will become immutable in fiona version 2.0
  feature['id'] = uuid.uuid4().int % (10**10)


## BLM lands 

Clean up BLM features

In [60]:
with fiona.open('sources/blm-all-lands.gpkg', 'r', layer='bounds') as src:
    blm_lands = [feature for feature in src]

    for feature in blm_lands:
        if feature['properties']['FeeTitleHolder'] == 'USDI Bureau of Land Management O&C Lands':
            name = 'Bureau of Land Management O&C Lands'
        else: 
            name = "Bureau of Land Management"
        feature['properties'] = {'owner': 'Bureau of Land Management', 'owner_class': 'federal', 'area': 0, 'percent_of_class': 0, 'name': name}
        
    # merge all features with the same name into a single feature
    grouped_blm_lands = {}
    for feature in blm_lands:
        if feature['properties']['name'] not in grouped_blm_lands:
            grouped_blm_lands[feature['properties']['name']] = feature
        else:
            geometry = shape(feature['geometry'])
            geometry2 = shape(grouped_blm_lands[feature['properties']['name']]['geometry'])
            combined_geometry = unary_union([geometry, geometry2])
            grouped_blm_lands[feature['properties']['name']] = {'geometry': combined_geometry, 'properties': feature['properties']}
            
    total_blm_area = 0
    # calculate the area of each feature
    for feature in grouped_blm_lands.values():
        feature['properties']['area'] = get_area_acres(shape(feature['geometry']))
        total_blm_area += feature['properties']['area']
        
    # add the percent of each feature
    for feature in grouped_blm_lands.values():
        feature['properties']['percent_of_class'] = round(feature['properties']['area'] / total_blm_area * 100, 2)
        
    schema = {
        'geometry': 'MultiPolygon',
        'id': 'int',
        'properties': {
            'owner': 'str',
            'owner_class': 'str',
            'area': 'int',
            'percent_of_class': 'float',
            'name': 'str'
        }
    }
   
    # save the grouped_blm_lands to a gpkg file
    with fiona.open('sources/clean-blm-lands.gpkg', 'w', driver='GPKG', schema=schema, crs=from_epsg(4326)) as dst:
        dst.writerecords(grouped_blm_lands.values())


/var/folders/8h/kk4k5t952ydbc8hfg6nct2940000gn/T/ipykernel_38736/1811175559.py:9: FionaDeprecationWarning: instances of this class -- CRS, geometry, and feature objects -- will become immutable in fiona version 2.0
  feature['properties'] = {'owner': 'Bureau of Land Management', 'owner_class': 'federal', 'area': 0, 'percent_of_class': 0, 'name': name}
